In [1]:
!pip install pulp
import pandas as pd
import random
import pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 21.4 MB/s eta 0:00:00


In [4]:
# データ準備
random.seed(0)
num_user = 24
departments = ['A部署', 'B部署', 'C部署', 'D部署', 'E部署']
services = ['Aサービス', 'Bサービス', 'Cサービス', 'Dサービス', 'Eサービス']
employee_ids = [f'社員{str(i+1).zfill(2)}' for i in range(num_user)]
data_with_experience = {
    '社員ID': employee_ids,
    '部署': [random.choice(departments) for _ in range(num_user)],
    '利用希望サービス': [random.sample(services, random.randint(1, 5)) for _ in range(num_user)],
    '分析経験年数': [random.randint(1, 3) for _ in range(num_user)],
    '研修参加フラグ': [random.choice([0, 1]) for _ in range(num_user)]
}
df = pd.DataFrame(data_with_experience)

# 特徴量
df['professional'] = (df['分析経験年数']>=3).astype(int)
df['beginner'] = (df['分析経験年数']==1).astype(int)
df

,社員ID,部署,利用希望サービス,分析経験年数,研修参加フラグ,professional,beginner
0,社員01,D部署,[Cサービス],1,0,0,1
1,社員02,D部署,"[Eサービス, Aサービス, Bサービス, Cサービス]",1,0,0,1
2,社員03,A部署,"[Eサービス, Bサービス, Cサービス]",1,0,0,1
3,社員04,C部署,"[Dサービス, Cサービス, Aサービス, Eサービス]",3,0,1,0
4,社員05,E部署,[Dサービス],1,1,0,1
5,社員06,D部署,[Eサービス],1,0,0,1
6,社員07,D部署,"[Cサービス, Bサービス, Eサービス, Dサービス]",3,0,1,0
7,社員08,C部署,[Bサービス],3,0,1,0
8,社員09,D部署,"[Bサービス, Eサービス, Aサービス, Dサービス, Cサービス]",3,1,1,0
9,社員10,C部署,[Cサービス],3,0,1,0


In [3]:
id_col = '社員ID'
department_col = '部署'
service_col = '利用希望サービス'

In [5]:
S = df[id_col].tolist()
C = ['Aチーム', 'Bチーム', 'Cチーム', 'Dチーム', 'Eチーム', 'Fチーム']

# 割り当て全候補のようなもの
SC = [(s,c) for s in S for c in C]

# 生徒をどのクラスに割り当てるかを変数として定義
x = pulp.LpVariable.dicts('x', SC, cat='Binary')

In [27]:
prob = pulp.LpProblem('AssignmentProblem', pulp.LpMaximize)
# 1.各社員は１つのクラスに割り当てる
for s in S:
    prob += pulp.lpSum([x[s,c] for c in C]) == 1

# 2.各チーム4~5人
for c in C:
    prob += pulp.lpSum([x[s,c] for s in S]) >=4
    prob += pulp.lpSum([x[s,c] for s in S]) <=5

# 3.同じ部署の人が2人以下
department_dict = {}
department_list = df[department_col].unique()
for d in department_list:
    department_dict[d] = df[df[department_col]==d][id_col].tolist()
for c in C:
    for d in department_list:
        prob += pulp.lpSum([x[s,c] for s in department_dict[d]]) <=2

# 4.経験者を1人以上
leader_list = df.query('professional==1')[id_col].tolist()
for c in C:
    prob += pulp.lpSum([x[s,c] for s in leader_list]) >=1

# 5.python初心者を1人以下
# beginner_list = df.query('beginner==1')[id_col].tolist()
# for c in C:
#     prob += pulp.lpSum([x[s,c] for s in beginner_list]) <=1

# 6.同じサービス希望者を集める
# Cチームの参加希望者が多いため2チームにしたい
s2se = {(s,c): 0 for s in S for c in C} # 人数*チーム数
service2team = {'Aサービス':'Aチーム', 'Bサービス':'Bチーム', 'Cサービス':'Cチーム',
                'Dサービス':'Dチーム', 'Eサービス':'Eチーム'}
for s, desired_services in zip(df[id_col].tolist(), df[service_col].tolist()):
    for d in desired_services:
        c = service2team[d]
        s2se[s, c] = 1
        if d=='Cサービス': # FチームもCサービス担当とする処理
            s2se[s, 'Fチーム'] = 1
prob += pulp.lpSum([x[s,c] * s2se[s,c] for s,c in SC])

In [28]:
# 求解
status = prob.solve()
print(status)
print(pulp.LpStatus[status])

1
Optimal


In [29]:
C2Ss = {}
for c in C:
    C2Ss[c] = [s for s in S if x[s,c].value()==1] # x[s,c]に割り当てだとvalueが1となる

for c, Ss in C2Ss.items():
    print('Class:', c)
    print('Num:', len(Ss))
    print('Student:', Ss)
    print()

Class: Aチーム
Num: 4
Student: ['社員02', '社員15', '社員19', '社員22']

Class: Bチーム
Num: 4
Student: ['社員03', '社員08', '社員18', '社員23']

Class: Cチーム
Num: 4
Student: ['社員10', '社員14', '社員16', '社員20']

Class: Dチーム
Num: 4
Student: ['社員05', '社員07', '社員09', '社員21']

Class: Eチーム
Num: 4
Student: ['社員06', '社員13', '社員17', '社員24']

Class: Fチーム
Num: 4
Student: ['社員01', '社員04', '社員11', '社員12']



In [30]:
# 確認
for c, Ss in C2Ss.items():
    display(df.query('社員ID==@Ss'))

,社員ID,部署,利用希望サービス,分析経験年数,研修参加フラグ,professional,beginner
1,社員02,D部署,"[Eサービス, Aサービス, Bサービス, Cサービス]",1,0,0,1
14,社員15,C部署,"[Bサービス, Aサービス]",3,1,1,0
18,社員19,C部署,"[Dサービス, Cサービス, Bサービス, Eサービス, Aサービス]",1,0,0,1
21,社員22,B部署,"[Aサービス, Cサービス]",2,0,0,0


,社員ID,部署,利用希望サービス,分析経験年数,研修参加フラグ,professional,beginner
2,社員03,A部署,"[Eサービス, Bサービス, Cサービス]",1,0,0,1
7,社員08,C部署,[Bサービス],3,0,1,0
17,社員18,E部署,"[Dサービス, Cサービス, Eサービス, Aサービス, Bサービス]",3,0,1,0
22,社員23,C部署,[Bサービス],1,1,0,1


,社員ID,部署,利用希望サービス,分析経験年数,研修参加フラグ,professional,beginner
9,社員10,C部署,[Cサービス],3,0,1,0
13,社員14,B部署,"[Cサービス, Bサービス, Dサービス, Aサービス]",1,0,0,1
15,社員16,B部署,"[Cサービス, Dサービス, Aサービス, Eサービス, Bサービス]",1,1,0,1
19,社員20,E部署,[Cサービス],2,1,0,0


,社員ID,部署,利用希望サービス,分析経験年数,研修参加フラグ,professional,beginner
4,社員05,E部署,[Dサービス],1,1,0,1
6,社員07,D部署,"[Cサービス, Bサービス, Eサービス, Dサービス]",3,0,1,0
8,社員09,D部署,"[Bサービス, Eサービス, Aサービス, Dサービス, Cサービス]",3,1,1,0
20,社員21,E部署,"[Aサービス, Dサービス, Cサービス, Bサービス, Eサービス]",1,0,0,1


,社員ID,部署,利用希望サービス,分析経験年数,研修参加フラグ,professional,beginner
5,社員06,D部署,[Eサービス],1,0,0,1
12,社員13,E部署,[Eサービス],1,0,0,1
16,社員17,A部署,"[Aサービス, Eサービス]",3,1,1,0
23,社員24,A部署,"[Bサービス, Cサービス, Eサービス]",1,0,0,1


,社員ID,部署,利用希望サービス,分析経験年数,研修参加フラグ,professional,beginner
0,社員01,D部署,[Cサービス],1,0,0,1
3,社員04,C部署,"[Dサービス, Cサービス, Aサービス, Eサービス]",3,0,1,0
10,社員11,E部署,"[Dサービス, Aサービス, Bサービス, Cサービス, Eサービス]",3,1,1,0
11,社員12,B部署,"[Cサービス, Bサービス, Eサービス, Dサービス, Aサービス]",1,0,0,1
